In [95]:
import mysql.connector

connection = mysql.connector.connect(
  host = "gateway01.ap-southeast-1.prod.aws.tidbcloud.com",
  port = 4000,
  user = "3GNYKqCbbJtp9c2.root",
  password = "XQYbmm6hIa6n0Hwg",
  database = "FirstDatabase",
)

if connection.is_connected():
    print("Connection successful")
    #connection.close()
    #print("Connection closed")
else:
    print("Connection Failed")

mycursor= connection.cursor()

Connection successful


In [ ]:
mycursor.execute("Select * from FirstDatabase.DiabetesData where outcome = 1 and (bmi between 18 and 25) ")
out = mycursor.fetchall()
from tabulate import tabulate
print(tabulate(out, headers=[i[0] for i in mycursor.description], showindex="always", tablefmt='psql'))

+----+---------------+-----------+-----------------+-----------------+-----------+-------+----------------------------+-------+-----------+
|    |   pregnancies |   glucose |   bloodpressure |   skinthickness |   insulin |   bmi |   diabetespedigreefunction |   age |   outcome |
|----+---------------+-----------+-----------------+-----------------+-----------+-------+----------------------------+-------+-----------|
|  0 |             8 |       183 |              64 |               0 |         0 |  23.3 |                      0.672 |    32 |         1 |
|  1 |             4 |       134 |              72 |               0 |         0 |  23.8 |                      0.277 |    60 |         1 |
|  2 |             3 |       107 |              62 |              13 |        48 |  22.9 |                      0.678 |    23 |         1 |
|  3 |             6 |       194 |              78 |               0 |         0 |  23.5 |                      0.129 |    59 |         1 |
|  4 |             1

In [72]:
# mycursor.execute("Select age, bmi from FirstDatabase.DiabetesData order by age desc, bmi desc limit 20")
#mycursor.execute("Select age, count(outcome) from FirstDatabase.DiabetesData group by age")
#mycursor.execute("Select age, count(outcome), avg(bmi) from FirstDatabase.DiabetesData where age < 30 group by outcome")
#mycursor.execute("Select count(pregnancies), pregnancies, outcome from FirstDatabase.DiabetesData group by pregnancies, outcome having count(*)>3 order by pregnancies desc")
#mycursor.execute("Select count(pregnancies), pregnancies, outcome from FirstDatabase.DiabetesData group by pregnancies, outcome order by pregnancies desc")
#mycursor.execute("Select count(pregnancies), pregnancies, outcome from FirstDatabase.DiabetesData group by pregnancies, outcome order by pregnancies desc")
# mycursor.execute('''SELECT count(*),
#        CASE
#            WHEN bmi < 18 THEN "underweight"
#            WHEN bmi BETWEEN 18 AND 25 THEN "Normal"
#            WHEN bmi BETWEEN 26 AND 30 THEN "overweight"
#            ELSE "obese"
#        END AS weight_category
# FROM FirstDatabase.DiabetesData
# group by weight_category;
# ''')

mycursor.execute("""
   SELECT Age, 
        Pregnancies, 
        BloodPressure,
        count(*) OVER 
        (PARTITION BY 
         Pregnancies 
         ORDER BY 
        BloodPressure DESC) as Rank
   FROM FirstDatabase.DiabetesData;
""")
out = mycursor.fetchall()
from tabulate import tabulate
print(tabulate(out, headers=[i[0] for i in mycursor.description], showindex="always", tablefmt='psql'))

ProgrammingError: 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your TiDB version for the right syntax to use line 9 column 36 near "Rank
   FROM FirstDatabase.DiabetesData;
" 

In [86]:
mycursor.execute("""SELECT
  `age`,
  `pregnancies`,
  `bloodpressure`,
   dense_rank() 
   
    OVER (
    PARTITION BY
      `pregnancies`
    ORDER BY
      `bloodpressure` DESC
  ) AS `Rank`
FROM
  FirstDatabase.DiabetesData;""")
out = mycursor.fetchall()
from tabulate import tabulate
print(tabulate(out, headers=[i[0] for i in mycursor.description], showindex="always", tablefmt='psql'))

+-----+-------+---------------+-----------------+--------+
|     |   age |   pregnancies |   bloodpressure |   Rank |
|-----+-------+---------------+-----------------+--------|
|   0 |    22 |             3 |             100 |      1 |
|   1 |    32 |             3 |              92 |      2 |
|   2 |    29 |             3 |              90 |      3 |
|   3 |    27 |             3 |              88 |      4 |
|   4 |    52 |             3 |              86 |      5 |
|   5 |    22 |             3 |              86 |      5 |
|   6 |    22 |             3 |              84 |      6 |
|   7 |    46 |             3 |              82 |      7 |
|   8 |    25 |             3 |              82 |      7 |
|   9 |    27 |             3 |              82 |      7 |
|  10 |    30 |             3 |              80 |      8 |
|  11 |    26 |             3 |              80 |      8 |
|  12 |    63 |             3 |              80 |      8 |
|  13 |    44 |             3 |              80 |      8

In [93]:
# mycursor.execute("""select count(*) from FirstDatabase.DiabetesData where outcome = 0 and 
#                  bmi in (select bmi from FirstDatabase.DiabetesData where bmi < 25)""")
mycursor.execute(""" 
select * from FirstDatabase.DiabetesData where bloodpressure > (select avg(bloodpressure) from FirstDatabase.DiabetesData)
""")
out = mycursor.fetchall()
from tabulate import tabulate
print(tabulate(out, headers=[i[0] for i in mycursor.description], showindex="always", tablefmt='psql'))

+-----+---------------+-----------+-----------------+-----------------+-----------+-------+----------------------------+-------+-----------+
|     |   pregnancies |   glucose |   bloodpressure |   skinthickness |   insulin |   bmi |   diabetespedigreefunction |   age |   outcome |
|-----+---------------+-----------+-----------------+-----------------+-----------+-------+----------------------------+-------+-----------|
|   0 |             6 |       148 |              72 |              35 |         0 |  33.6 |                      0.627 |    50 |         1 |
|   1 |             5 |       116 |              74 |               0 |         0 |  25.6 |                      0.201 |    30 |         0 |
|   2 |             2 |       197 |              70 |              45 |       543 |  30.5 |                      0.158 |    53 |         1 |
|   3 |             8 |       125 |              96 |               0 |         0 |   0   |                      0.232 |    54 |         1 |
|   4 |      

In [96]:
mycursor.execute("Create table FirstDatabase.StudentsDB_1 (no Integer, Name Text NOT NULL, school varchar(20))")

In [102]:
query = "Insert into FirstDatabase.StudentsDB_1 (no, Name, school) values (%s, %s, %s)"
mycursor.execute(query, (None, 'xyz', 'abc'))
connection.commit()

In [103]:
mycursor.execute(""" 
select * from FirstDatabase.StudentsDB_1
""")
out = mycursor.fetchall()
from tabulate import tabulate
print(tabulate(out, headers=[i[0] for i in mycursor.description], showindex="always", tablefmt='psql'))

+----+------+--------+----------+
|    |   no | Name   | school   |
|----+------+--------+----------|
|  0 |    1 | abc    | abc      |
|  1 |    1 | abc    | abc      |
|  2 |      | xyz    | abc      |
+----+------+--------+----------+
